In [ ]:
# faire les importations nécessaires

import psycopg
from psycopg import Cursor
import ollama   # <-- Correct

# Déclarer les variables nécessaires
conversation_file_path = "../data/017_00000012.txt"   # <-- corrige le chemin si besoin

# Le modèle d'embeddings Ollama
EMBED_MODEL = "embeddinggemma"

db_connection_str = "dbname=rag_chatbot user=postgres password=1803 host=localhost port=5432"


def create_conversation_list(file_path: str) -> list[str]:
    """Lit un fichier et filtre les lignes inutiles."""
    with open(file_path, "r", encoding="latin-1") as file:
        text_list = file.read().split("\n")

    filtered_list = [
        line.removeprefix("     ")
        for line in text_list
        if not line.startswith("<") and line.strip() != ""
    ]

    print(filtered_list)
    return filtered_list


def calculate_embeddings(corpus: str) -> list[float]:
    response = ollama.embeddings(EMBED_MODEL, corpus)
    return response["embedding"]

def to_pgvector(vec: list[float]) -> str:
    return "[" + ",".join(str(v) for v in vec) + "]"


def save_embedding(corpus: str, embedding: list[float], cursor: Cursor) -> None:
    pg_vec = to_pgvector(embedding)
    cursor.execute(
        """
        INSERT INTO embeddings (corpus, embedding)
        VALUES (%s, %s::vector)
        """,
        (corpus, pg_vec),
    )



def similar_corpus(input_corpus: str, k: int, cursor: Cursor):
    embedding = calculate_embeddings(input_corpus)
    pg_vec = to_pgvector(embedding)

    cursor.execute(
        """
        SELECT id, corpus, embedding <=> %s::vector AS distance
        FROM embeddings
        ORDER BY distance ASC
        LIMIT %s
        """,
        (pg_vec, k),
    )

    return cursor.fetchall()


# ------------------------------
# Création + insertion embeddings
# ------------------------------

with psycopg.connect(db_connection_str) as conn:
    conn.autocommit = True

    with conn.cursor() as cur:
        # Supprimer ancienne table
        cur.execute("DROP TABLE IF EXISTS embeddings")

        # Créer extension pgvector
        cur.execute("CREATE EXTENSION IF NOT EXISTS vector")

        # Vérifier la dimension réelle du modèle
        # embeddinggemma = 768 dimensions
        cur.execute(
            """
            CREATE TABLE IF NOT EXISTS embeddings (
                id SERIAL PRIMARY KEY,
                corpus TEXT,
                embedding VECTOR(768)
            );
            """
        )

        # Charger les corpus
        corpus_list = create_conversation_list(conversation_file_path)

        # Calcul embeddings + insertion
        for corpus in corpus_list:
            emb = calculate_embeddings(corpus)
            save_embedding(corpus, emb, cur)

        conn.commit()

        # Test similarité
        print("\n--- Résultats similarité (test) ---")
        results = similar_corpus("bonjour je veux de l'aide", 3, cur)
        for r in results:
            print(r)


['h: U B S bonjour', "c: oui bonjour e j'appelle je sais pas si j'appelle au bon endroit e", 'h: je vous écoute', "c: c'est pour", "c: e c'est pour savoir si la fac pendant l'été e a des professeurs ou des des gens qui font des stages de de perfectionnement en anglais et en espagnol", 'h: e ce serait pour vous vous souhaiteriez', 'h: non', "c: non non c'est pas pour moi", 'c: ce serait pour ma fille', 'h: oui', 'c: mais bon elle est', 'c: en seconde et', 'h: et elle souhaiterais se perfectionner', 'c: bon elle va passer en première mais', "h: en anglais ou en espagnol pendant l'été", "c: ouais c'est çà", 'h: oui alors e la fac de e de lettre et de langues se trouve à Lorient donc il faudrait plutôt  voir avec Lorient pour e savoir si ils organisent des stages mais en tout cas fac est fermée du 23 juillet au 23 août', "c: ouais par contre vous pas me m'orienter vers e", 'h: vers Lorient', 'c: e', "c: un organisme sur Vannes qui qui s'occupe de ce genre de chose", 'h: e non à priori non 